In [8]:
import sys
sys.path.append("/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/reproduce/model/")


In [9]:
import numpy as np
import torch

from hfnet.evaluation.keypoint_detectors import compute_average_precision, compute_correctness, compute_pr
from hfnet.evaluation.utils.keypoints import keypoints_warp_2D
from hfnet.evaluation.utils.misc import div0
import tqdm

from torch.utils.data import DataLoader

from hfnet.dataset.hpatches import HpatchesDataset


from hfnet.model import HFNet

from hfnet import prediction
from hfnet import evaluation
from hfnet.export_local import export_loader

In [10]:
# Load dataset
dataset = HpatchesDataset(alteration='all', make_pairs=True, shuffle=True)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)


mode_config= {
        'loss_weights': 'uncertainties',
        'local_head': {
            'descriptor_dim': 256,
            'detector_grid': 8,
            'input_channels': 96
        },
        'global_head': {
            'n_clusters': 32,
            'intermediate_proj': 0,
            'dimensionality_reduction': 4096
        }
}
model = HFNet(config= mode_config,width_mult=0.75)


In [ ]:
EXPER_PATH = '/media/tupv8/32f7b466-e4d4-4f6f-b2aa-1e5a8ce7663c/vinai/reproduce/model/'

models = { 
    'silk':{
        'binary_cross_entropy' : 'silk/111/binary_cross_entropy/last_model.pth',
        'cross_entropy': 'silk/111/cross_entropy/last_model.pth',
    }
}

config = {
    'hfnet':{
        'local':{
            'nms_radius':4,
            'detector_threshold': 0.005,
            'num_keypoints': 1000,
        },
        'num_features': 300,
        # 'do_nms': True,
        # 'nms_thresh': 4,
        # 'remove_borders': 4,
    },
    'do_ratio_test': False,
    'correct_match_thresh': 3,
    'correct_H_thresh': 3,
}

In [12]:
def evaluate(model, dataloader, config):
    num_kpts = []
    loc_error = []
    repeatability = []
    all_tp = []
    all_num_gt = 0
    all_scores = []

    for data in tqdm.tqdm(dataloader):
        
        shape1 = data['image'].detach().cpu().numpy().shape[-2:][::-1]  # (height, width) -> (width, height)
        shape2 = data['image2'].detach().cpu().numpy().shape[-2:][::-1]  # modifiy

        #print(shape1,shape2)

        pred1 = export_loader(model(data['image'].unsqueeze(0)), config['hfnet'], data['image'].squeeze(0))
        pred2 = export_loader(model(data['image2'].unsqueeze(0)), config['hfnet'], data['image2'].squeeze(0))


        num_kpts.extend([len(pred1['keypoints']), len(pred2['keypoints'])])
        if len(pred1['keypoints']) == 0 or len(pred2['keypoints']) == 0:
            repeatability.append(0)
            continue
        H = data['homography'][0]
        #print(H)
        kpts1_w, vis1 = keypoints_warp_2D(
            pred1['keypoints'], np.linalg.inv(H), shape2)
        kpts2_w, vis2 = keypoints_warp_2D(
            pred2['keypoints'], H, shape1)


        correct1, correct2, dist1, dist2 = compute_correctness(
            pred1['keypoints'], pred2['keypoints'], kpts1_w, kpts2_w,
            config['correct_match_thresh'])

        error1 = dist1[vis1 & correct1]
        if len(error1) > 0:
            loc_error.append(error1.mean())
        error2 = dist2[vis2 & correct2]
        if len(error2) > 0:
            loc_error.append(error2.mean())

        repeat = div0(correct1[vis1].sum() + correct2[vis2].sum(),
                        vis1.sum() + vis2.sum())
        repeatability.append(repeat)

        all_tp.extend([correct1[vis1], correct2[vis2]])
        all_scores.extend([pred1['scores'][vis1], pred2['scores'][vis2]])
        all_num_gt += vis2.sum() + vis1.sum()

    precision, recall, scores = compute_pr(
        np.concatenate(all_tp, 0), np.concatenate(all_scores, 0), all_num_gt,
        reverse=True)  # confidence is in decreasing order
    mAP = compute_average_precision(precision, recall)

    metrics = {
        'average_num_keypoints': np.mean(num_kpts),
        'localization_error': np.mean(loc_error),
        'repeatability': np.mean(repeatability),
        'mAP': mAP,
    }
    return metrics


In [13]:
import time

device = torch.device('cpu')
model.to(device)

for k,v in models.items():
    for kk,vv in v.items():
        print(f"{k}: {kk}")
        print(config)
        state_dict = torch.load(EXPER_PATH + vv, weights_only= False ,map_location=device)
        model.load_state_dict(state_dict, strict=False)

        metrics=evaluate(model, dataloader, config)
        

        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(current_time)

        log_file = open('detector.txt', 'a')
        log_file.write('\n[{}]'.format(current_time))
        log_file.write('\n{}: {}'.format(k, kk))
        log_file.write('{}\n'.format(config))


        for km, vm in metrics.items():
                print('{:<25} {:.3f}'.format(km, vm))

                #save in log file
                log_file.write('{:<25} :{:.3f}  '.format(km, vm))
                log_file.write('\n')
        log_file.flush()
        print("-----------------------------------")

       

superpoint: batch 1
{'hfnet': {'local': {'nms_radius': 4, 'detector_threshold': 0.005, 'num_keypoints': 1000}, 'num_features': 300}, 'do_ratio_test': False, 'correct_match_thresh': 3, 'correct_H_thresh': 3}


100%|██████████| 580/580 [09:51<00:00,  1.02s/it]


2025-03-20 10:34:00
average_num_keypoints     300.000
localization_error        1.580
repeatability             0.217
mAP                       0.064
-----------------------------------
